## Formulación del Problema
Salon de belleza "Los espejos" cuenta con dos estilistas que se demoran habitualmente entre 20 a 35 minutos por corte. 

Los clientes son atendidos de acuerdo al orden de llegada.

La peluquería recibe en promedio 3 clientes por hora (es decir, uno cada 20 minutos). 

Se desea simular las llegadas y servicios de 30 clientes, que es lo que se atiende aproximadamente en una jornada regular de trabajo. 



In [2]:
!pip install simpy

In [0]:
#importando librerias
import random
import math
import simpy

In [0]:
#declaracion de variables
SEMILLA = 30
NUM_PELUQUEROS = 2
TIEMPO_CORTE_MIN = 20
TIEMPO_CORTE_MAX = 35
T_LLEGADAS = 20
TIEMPO_SIMULACION = 120
TOT_CLIENTES = 30
 
te  = 0.0 # tiempo de espera total
dt  = 0.0 # duracion de servicio total
fin = 0.0 # minuto en el que finaliza

In [0]:
#Definicion de Procedimientos
def cortar(cliente):
	global dt  #Para poder acceder a la variable dt declarada anteriormente
	R = random.random()  # Obtiene un numero aleatorio y lo guarda en R
	tiempo = TIEMPO_CORTE_MAX - TIEMPO_CORTE_MIN  
	tiempo_corte = TIEMPO_CORTE_MIN + (tiempo*R) # Distribucion uniforme
	yield env.timeout(tiempo_corte) # deja correr el tiempo n minutos
	print(" \o/ Corte listo a %s en %.2f minutos" % (cliente,tiempo_corte))
	dt = dt + tiempo_corte # Acumula los tiempos de uso de la i
  
def cliente (env, name, personal ):
	global te
	global fin
	llega = env.now # Guarda el minuto de llegada del cliente
	print ("---> %s llego a peluqueria en minuto %.2f" % (name, llega))
	with personal.request() as request: # Espera su turno
		yield request # Obtiene turno
		pasa = env.now # Guarda el minuto cuado comienza a ser atendido
		espera = pasa - llega # Calcula el tiempo que espero
		te = te + espera # Acumula los tiempos de espera
		print ("**** %s pasa con peluquero en minuto %.2f habiendo esperado %.2f" % (name, pasa, espera))
		yield env.process(cortar(name)) # Invoca al proceso cortar
		deja = env.now #Guarda el minuto en que termina el proceso cortar 
		print ("<--- %s deja peluqueria en minuto %.2f" % (name, deja))
		fin = deja # Conserva globalmente el ultimo minuto de la simulacion
	 
def principal (env, personal):
	llegada = 0
	i = 0
	for i in range(TOT_CLIENTES): # Para n clientes
		R = random.random()
		llegada = -T_LLEGADAS * math.log(R) # Distribucion exponencial
		yield env.timeout(llegada)  # Deja transcurrir un tiempo entre uno y otro
		i += 1
		env.process(cliente(env, 'Cliente %d' % i, personal)) 

In [14]:
#ejecucion de la simulacion
print ("------------------- Iniciando Simulacion ------------------")
random.seed (SEMILLA)  # Valor semilla random
env = simpy.Environment() # Crea el objeto entorno de simulacion
personal = simpy.Resource(env, NUM_PELUQUEROS) #Crea los recursos (peluqueros)
env.process(principal(env, personal)) #Invoca el proceso princial
env.run() #Inicia la simulacion

------------------- Bienvenido Simulacion Peluqueria ------------------
---> Cliente 1 llego a peluqueria en minuto 12.36
**** Cliente 1 pasa con peluquero en minuto 12.36 habiendo esperado 0.00
 \o/ Corte listo a Cliente 1 en 20.30 minutos
<--- Cliente 1 deja peluqueria en minuto 32.66
---> Cliente 2 llego a peluqueria en minuto 37.17
**** Cliente 2 pasa con peluquero en minuto 37.17 habiendo esperado 0.00
---> Cliente 3 llego a peluqueria en minuto 45.67
**** Cliente 3 pasa con peluquero en minuto 45.67 habiendo esperado 0.00
 \o/ Corte listo a Cliente 2 en 22.10 minutos
<--- Cliente 2 deja peluqueria en minuto 59.27
 \o/ Corte listo a Cliente 3 en 23.97 minutos
<--- Cliente 3 deja peluqueria en minuto 69.65
---> Cliente 4 llego a peluqueria en minuto 72.83
**** Cliente 4 pasa con peluquero en minuto 72.83 habiendo esperado 0.00
---> Cliente 5 llego a peluqueria en minuto 81.70
**** Cliente 5 pasa con peluquero en minuto 81.70 habiendo esperado 0.00
---> Cliente 6 llego a peluqueria 

In [15]:
#Calculo de indicadores
print ("\n---------------------------------------------------------------------")
print ("\nIndicadores obtenidos: ")
 
lpc = te / fin
print ("\nLongitud promedio de la cola: %.2f" % lpc)
tep = te / TOT_CLIENTES
print ("Tiempo de espera promedio = %.2f" % tep)
upi = (dt / fin) / NUM_PELUQUEROS
print ("Uso promedio de la instalacion = %.2f" % upi)
print ("\n---------------------------------------------------------------------")


---------------------------------------------------------------------

Indicadores obtenidos: 

Longitud promedio de la cola: 0.50
Tiempo de espera promedio = 7.85
Uso promedio de la instalacion = 0.80

---------------------------------------------------------------------


##Conclusión

La simulación en Python usando Simpy es una valiosa herramienta para obtener resultados en forma más sencilla y poder optimizar los procesos.